In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install gensim
import os
import gensim

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# file_wiki = '/content/drive/MyDrive/embedding/idwiki-latest-pages-articles.xml.bz2'
# file_idwiki = '/content/drive/MyDrive/embedding/idwiki.txt'

In [ ]:
# if not os.path.isfile(file_wiki):
#     !wget https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2

--2023-08-02 13:17:53--  https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870286461 (830M) [application/octet-stream]
Saving to: ‘idwiki-latest-pages-articles.xml.bz2’

idwiki-latest-pages 100%[===================>] 829.97M  4.36MB/s    in 3m 13s  

2023-08-02 13:21:07 (4.29 MB/s) - ‘idwiki-latest-pages-articles.xml.bz2’ saved [870286461/870286461]



In [ ]:
# wiki_indo = gensim.corpora.WikiCorpus(file_wiki, dictionary={})

In [ ]:
# with open(file_idwiki, 'w') as f:
#     i=0
#     for text in wiki_indo.get_texts():
#         text= ' '.join(text)
#         text= text.lower()
#         f.write(text + '\n')
#         i += 1
#         print(str(i), 'articles processed')

#     print('total', str(i))

Output streaming akan dipotong hingga 5000 baris terakhir.
485738 articles processed
485739 articles processed
485740 articles processed
485741 articles processed
485742 articles processed
485743 articles processed
485744 articles processed
485745 articles processed
485746 articles processed
485747 articles processed
485748 articles processed
485749 articles processed
485750 articles processed
485751 articles processed
485752 articles processed
485753 articles processed
485754 articles processed
485755 articles processed
485756 articles processed
485757 articles processed
485758 articles processed
485759 articles processed
485760 articles processed
485761 articles processed
485762 articles processed
485763 articles processed
485764 articles processed
485765 articles processed
485766 articles processed
485767 articles processed
485768 articles processed
485769 articles processed
485770 articles processed
485771 articles processed
485772 articles processed
485773 articles processed
48577

In [ ]:
# sentences_wiki = gensim.models.word2vec.LineSentence(file_idwiki)
# model = gensim.models.word2vec.Word2Vec(sentences_wiki, vector_size = 100,window = 5,min_count=1, workers=4,sg=0)
# model.save(model_path)

In [ ]:
model_path = '/content/drive/MyDrive/embedding/wiki_word2vec_gensim.model'

In [ ]:
from gensim.models import Word2Vec



word2vec_model = Word2Vec.load(model_path)



In [ ]:
! pip install keras_preprocessing
! pip install sastrawi
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
df = pd.read_excel('/content/UTSDataset.xlsx')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout,SpatialDropout1D,Reshape
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras import regularizers
import re
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# nltk.download('stopwords')

factory = StemmerFactory()
stemmer = factory.create_stemmer()

stopwords = nltk.corpus.stopwords.words('indonesian')
# Membaca dataset dari file CSV
# df = pd.read_excel('dataset coba uts.xlsx')

# NO 1
# Memisahkan kolom jawaban dan label

df['Jawaban 2'] = df['Jawaban 2'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['Jawaban 2'] = df['Jawaban 2'].str.lower()
df['Jawaban 2'] = df['Jawaban 2'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split() if word not in stopwords]))

jawaban = df['Jawaban 2'].values
label = df['nilai 2'].values

# Tokenisasi teks jawaban
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(jawaban)

#fungsi memberi kata salah ketika ada array yang kosong saat stemming
def handle_empty_array(tokens_array):
    if not tokens_array:  # Jika array kosong setelah stemming
        return ['salah']  # Mengganti dengan array berisi kata "salah"
    return tokens_array

tokenized = []
for d in jawaban:
    tokens = nltk.word_tokenize(d.lower())
    tokenized.append(tokens)
tokenized = [handle_empty_array(tokens) for tokens in tokenized]
tokenss = np.array(tokenized)

<ipython-input-17-0efb2a57e59f>:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tokenss = np.array(tokenized)


In [ ]:
# Konversi teks ke vektor numerik
jawaban_sequences = []
for sentence in tokenized:
    sequence = []
    for word in sentence:
        if word in word2vec_model.wv:
            sequence.append(word2vec_model.wv[word])
        else:
            sequence.append(np.zeros((100)))
    jawaban_sequences.append(sequence)

In [ ]:
# Menentukan panjang maksimum sekuens vektor
max_length = 100

# Padding sequence agar memiliki panjang yang sama
jawaban_padded = []

for i,v in enumerate(jawaban_sequences):
    vector = []
    if len(v) < max_length:
        pad_count = max_length - len(v)
        pad = np.zeros((pad_count, 100))
        vector = np.append(v, pad, axis = 0)
    else:
        vector = v[:max_length]
    jawaban_padded.append(vector)

# Konversi jawaban_padded menjadi np.array
jawaban_padded = np.array(jawaban_padded)

# Menampilkan hasil padding
print(jawaban_padded.shape)


(31, 100, 100)


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop

# Input data shapes
input_shape = (jawaban_padded.shape[1], jawaban_padded.shape[2])

# Preprocess data and labels
indices = np.arange(len(label))
indices_train, indices_test, y_train, y_test, index_train, index_test = train_test_split(indices, label, indices, test_size=0.8, random_state=42)
X_train, X_test = jawaban_padded[indices_train], jawaban_padded[indices_test]
y_train = to_categorical(y_train/10, 11)
y_test = to_categorical(y_test/10, 11)

# Model architecture
model = Sequential()
model.add(Conv1D(150, 5, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(100, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(3, 3, activation='relu'))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

model.add(Flatten())
model.add(Dense(3,activation='relu')) #hidden layer
model.add(Dense(2,activation='relu')) #hidden laye
model.add(Dense(11,activation='softmax'))

# Set the learning rate
learning_rate = 0.01
optimizer = RMSprop(learning_rate=learning_rate)

# Model compilation
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=32)

# Model evaluation
loss, acc = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', acc)

# Predictions
y_pred = model.predict(X_test)

for i in range(len(y_pred)):
    original_index = indices_test[i]
    nama = df.loc[original_index, 'Nama']
    nim = df.loc[original_index, 'NIM']
    nilai_real = np.argmax(y_test[i, :])
    nilai_prediksi = np.argmax(y_pred[i])

    print('Nama Mahasiswa:', nama)
    print('NIM Mahasiswa:', nim)
    print('Nilai Real:', nilai_real)
    print('Nilai Prediksi:', nilai_prediksi)

# Plotting the training loss and accuracy
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

from sklearn.metrics import classification_report
# Model evaluation
loss, acc = model.evaluate(X_test, y_test)
print('Loss val:', loss)
print('Accuracy val:', acc)
# Model evaluation
loss, acc = model.evaluate(X_train, y_train)
print('Loss :', loss)
print('Accuracy:', acc)

# Predictions
y_pred = model.predict(X_test)

# Convert one-hot encoded y_pred and y_test to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Classification report
report = classification_report(y_test_labels, y_pred_labels)
print('Classification Report:')
print(report)

# Predictions
y_pred = model.predict(X_test)

# Convert one-hot encoded y_pred and y_test to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Classification report
report = classification_report(y_test_labels, y_pred_labels)
print('Classification Report:')
print(report)

# Menampilkan confusion matrix dalam bentuk heatmap
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming you have predictions and true labels as y_pred_labels and y_test_labels respectively
cm = confusion_matrix(y_test_labels, y_pred_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()




In [ ]:
word2vec_model.wv.similarity('hitam', 'air')

0.23088421

In [ ]:
word2vec_model.wv.similar_by_word('raja')

[('penguasa', 0.7823464870452881),
 ('kerajaan', 0.7719171643257141),
 ('rajanya', 0.7586086988449097),
 ('maharaja', 0.7410613894462585),
 ('firaun', 0.7158430218696594),
 ('ratu', 0.7130628228187561),
 ('ratunya', 0.6792208552360535),
 ('pangeran', 0.6772819757461548),
 ('kertanagara', 0.6555689573287964),
 ('permaisurinya', 0.6506747007369995)]